In [1]:
# import libraries
from math import * 
import math
import sympy as sy
import numpy as np
import pandas as pd
import matplotlib as mpl
%matplotlib widget
import trackpy as tp
import os
import pickle
import re

from scipy import integrate
from scipy import signal
from scipy.signal import find_peaks
from numpy.linalg import norm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.fft import fft, fftfreq
from scipy import optimize
from sklearn.cluster import DBSCAN
from matplotlib.widgets import SpanSelector
from scipy.integrate import simpson
from numpy import trapz

# odr function from scipy package
# is used to perform ODR regression
from scipy import odr  
from scipy.optimize import leastsq


# --> Change globally the fonts of the plots:
plt.rcParams.update({'font.size': 12})
plt.rcParams.update({ 'font.sans-serif':'Arial'})
# to restore defaults use:    mpl.rc_file_defaults()

In [2]:
# Used function in the program:
def filt(df, var, low, high):
    #filter function
    return df[(df[var]>=low) & (df[var]<=high)]


# make a grid
def grid (start,end):
    minor_ticks=np.linspace(start,end,end+1)
    ax.set_xticks(minor_ticks,minor=True)
    ax.set_yticks(minor_ticks,minor=True)
    
    major_ticks = np.linspace(start,int(end),int(end/5+1))
    ax.set_yticks(major_ticks,major=True)
    ax.set_xticks(major_ticks,major=True)
    
    ax.grid(which="minor",alpha=0.3)
    ax.grid(which = 'major',color = 'green', linestyle = '--', linewidth = 0.5)
    
# Used sites:
# https://en.wikipedia.org/wiki/Rotation_matrix#Rotation_matrix_from_axis_and_angle
# https://www.matheboard.de/archive/598910/thread.html


def calc_average(l):
    l = [x for x in l if str(x) != 'nan']
    if len(l) > 0:
        return np.mean(l)
    else:
        return 0
      
def plot_D_band(ax,y):
    real_D_band = np.array(np.linspace(67,67*7,7))
    sb =  real_D_band - 5
    eb =  real_D_band + 5
    borders = [[sb[j],eb[j]] for j in range(len(real_D_band))] 
    return [ax.fill_between(j,max(y),min(y),color = 'green',alpha = .2) for j in borders]

def find_strings_containing_substring(strings, substring):
    return [s for s in strings if substring in s]

def resize_img(img, h, w):
    hh=int(img.shape[0]/h)
    ww=int(img.shape[1]/w)
    imag=np.zeros([h, w])
    for i in range(h):
        for j in range(w):
            imag[i,j]=np.sum(img[i*hh:(i+1)*hh, j*ww:(j+1)*ww])

def psf_img_rec2(psf, df):
    x=(df['Y']).values.astype(int) # 10 nm/pixel
    y=(df['X']).values.astype(int)
    #z=df['z_cor'].values
    #N=df['N'].values
    s=psf.shape
    image=np.zeros([2000,2000])
    for i in range(len(df)):
        image[int(np.round(x[i]-(26))):int(np.round(x[i]+(25))) , 
              int(np.round(y[i]-(26))):int(np.round(y[i]+(25)))]+=psf
    return image

# Function to read CSV files with error handling
def read_csv_file(file_name):
    try:
        return pd.read_csv(file_name)
    except FileNotFoundError:
        print(f"File not found: {file_name}")
        return None
    
def cross(ax): # used for plotting
    ax.axhline(y=0, xmin=0, xmax=1,color ='lightgrey')
    ax.axvline(x=0, ymin=0, ymax=1,color ='lightgrey') 
    
# Taken from https://scipython.com/blog/direct-linear-least-squares-fitting-of-an-ellipse/

def fit_ellipse(x, y):
    """

    Fit the coefficients a,b,c,d,e,f, representing an ellipse described by
    the formula F(x,y) = ax^2 + bxy + cy^2 + dx + ey + f = 0 to the provided
    arrays of data points x=[x1, x2, ..., xn] and y=[y1, y2, ..., yn].

    Based on the algorithm of Halir and Flusser, "Numerically stable direct
    least squares fitting of ellipses'.


    """

    D1 = np.vstack([x**2, x*y, y**2]).T
    D2 = np.vstack([x, y, np.ones(len(x))]).T
    S1 = D1.T @ D1
    S2 = D1.T @ D2
    S3 = D2.T @ D2
    T = -np.linalg.inv(S3) @ S2.T
    M = S1 + S2 @ T
    C = np.array(((0, 0, 2), (0, -1, 0), (2, 0, 0)), dtype=float)
    M = np.linalg.inv(C) @ M
    eigval, eigvec = np.linalg.eig(M)
    con = 4 * eigvec[0]* eigvec[2] - eigvec[1]**2
    ak = eigvec[:, np.nonzero(con > 0)[0]]
    return np.concatenate((ak, T @ ak)).ravel()


def cart_to_pol(coeffs):
    """

    Convert the cartesian conic coefficients, (a, b, c, d, e, f), to the
    ellipse parameters, where F(x, y) = ax^2 + bxy + cy^2 + dx + ey + f = 0.
    The returned parameters are x0, y0, ap, bp, e, phi, where (x0, y0) is the
    ellipse centre; (ap, bp) are the semi-major and semi-minor axes,
    respectively; e is the eccentricity; and phi is the rotation of the semi-
    major axis from the x-axis.

    """

    # We use the formulas from https://mathworld.wolfram.com/Ellipse.html
    # which assumes a cartesian form ax^2 + 2bxy + cy^2 + 2dx + 2fy + g = 0.
    # Therefore, rename and scale b, d and f appropriately.
    a = coeffs[0]
    b = coeffs[1] / 2
    c = coeffs[2]
    d = coeffs[3] / 2
    f = coeffs[4] / 2
    g = coeffs[5]

    den = b**2 - a*c
    if den > 0:
        raise ValueError('coeffs do not represent an ellipse: b^2 - 4ac must'
                         ' be negative!')

    # The location of the ellipse centre.
    x0, y0 = (c*d - b*f) / den, (a*f - b*d) / den

    num = 2 * (a*f**2 + c*d**2 + g*b**2 - 2*b*d*f - a*c*g)
    fac = np.sqrt((a - c)**2 + 4*b**2)
    # The semi-major and semi-minor axis lengths (these are not sorted).
    ap = np.sqrt(num / den / (fac - a - c))
    bp = np.sqrt(num / den / (-fac - a - c))

    # Sort the semi-major and semi-minor axis lengths but keep track of
    # the original relative magnitudes of width and height.
    width_gt_height = True
    if ap < bp:
        width_gt_height = False
        ap, bp = bp, ap

    # The eccentricity.
    r = (bp/ap)**2
    if r > 1:
        r = 1/r
    e = np.sqrt(1 - r)

    # The angle of anticlockwise rotation of the major-axis from x-axis.
    if b == 0:
        phi = 0 if a < c else np.pi/2
    else:
        phi = np.arctan((2.*b) / (a - c)) / 2
        if a > c:
            phi += np.pi/2
    if not width_gt_height:
        # Ensure that phi is the angle to rotate to the semi-major axis.
        phi += np.pi/2
    phi = phi % np.pi

    return x0, y0, ap, bp, e, phi


def get_ellipse_pts(params, npts=10000, tmin=0, tmax=2*np.pi):
    """
    Return npts points on the ellipse described by the params = x0, y0, ap,
    bp, e, phi for values of the parametric variable t between tmin and tmax.

    """

    x0, y0, ap, bp, e, phi = params
    # A grid of the parametric variable, t.
    t = np.linspace(tmin, tmax, npts)
    x = x0 + ap * np.cos(t) * np.cos(phi) - bp * np.sin(t) * np.sin(phi)
    y = y0 + ap * np.cos(t) * np.sin(phi) + bp * np.sin(t) * np.cos(phi)
    return x, y


def calc_height(locS_c,upper_cutoff=None,def_para=None):
    locS_c = locS_c.sort_values('X')
    
    # Set default value inside the function
    if upper_cutoff is None:
        upper_cutoff = max(locS_c['Y']) / 2
        
#     upper_cutoff1 = np.mean(locS_c['Y'])
#     upper_cutoff2 = max(locS_c['Y'])/2

#     if upper_cutoff1 < upper_cutoff2:
#         upper_cutoff = upper_cutoff2
#     else:
#         upper_cutoff = upper_cutoff1

    STORM_Data_upper_half = locS_c[locS_c['Y']>upper_cutoff]

    # make the fit for the upper data points
    upper_fit = np.polyfit(STORM_Data_upper_half['X'],STORM_Data_upper_half['Y'],2)
    x_upper_fit = np.linspace(-100,100,1000)
    y_upper_fit = np.polyval(upper_fit,x_upper_fit)

    # save the upper point:
    height_polyfit = max(y_upper_fit)
    x_STORM_fit_max = x_upper_fit[list(y_upper_fit).index(max(y_upper_fit))]



    # Binning the Data like in AFM:
    Data_n_STORM = filt(locS_c,'X',-120,120)
    Data_n_STORM = Data_n_STORM.sort_values('X')

    # bin the data along the X-axis
    X_range = max(Data_n_STORM['X']) + abs(min(Data_n_STORM['X']))
    bin_width = 20

    num_of_bins = int(X_range/bin_width)
    c_data = pd.cut(Data_n_STORM['X'],bins=num_of_bins) # bins with equal width
    Data_n_STORM['bin'] = c_data
    Data_n_STORM_mean = Data_n_STORM.groupby(['bin']).mean()
    Data_n_STORM_mean = Data_n_STORM_mean.sort_values('X')

    STORM_mean_height = list(Data_n_STORM_mean['Y'])

    # Height takes the max of the binned data points
    height_bin = round(max(STORM_mean_height),2)
    x_STORM_fib_height = list(Data_n_STORM_mean['X'])[STORM_mean_height.index(max(STORM_mean_height))]


    STORM_sw_H_n,x_max_sw_n,y_max_sw_n = sliding_win_n(locS_c,n_ws,overlap_n)
    sliding_window_height = y_max_sw_n
    # sliding window binning datapoints
    
    
    # Simple Height approach:
    fig_STORM_cross_section_topfit, ax1=plt.subplots()
    ax1.scatter(STORM_Data_upper_half['X'],STORM_Data_upper_half['Y'],s=0.8,alpha=0.9,c='b') 
    ax1.scatter(locS_c['X'],locS_c['Y'], s=0.5, alpha=0.8,c='grey')
    ax1.plot(x_upper_fit,y_upper_fit)
    ax1.plot([x_STORM_fit_max,x_STORM_fit_max],[0,height_polyfit],color='blue',
                label = f'height_polyfit: {round(height_polyfit,2)}')
    ax1.set_xlabel("X / nm") # X-axis label
    ax1.set_ylabel("Z / nm") # Y-axis label
    ax1.axis('equal')

    # ax1.plot([-200,200],[upper_cutoff1,upper_cutoff1],c='magenta')
    # ax1.plot([-200,200],[upper_cutoff2,upper_cutoff2],c='cyan')

    ax1.scatter(Data_n_STORM_mean['X'], STORM_mean_height,color='r',s=10) 
    ax1.plot([x_STORM_fib_height,x_STORM_fib_height],[0,height_bin],
             color='r',label=f'binned height: {height_bin} nm')
    ax1.axhline(color='k',alpha=0.2)
    ax1.axvline(color='k',alpha=0.2)
    
    # Plot the sliding window binning:
    ax1.scatter(STORM_sw_H_n[0],STORM_sw_H_n[1],c='g',marker='+',
            label = f'Nr of datapoints per bin: {n_ws},'+
            f' overlap: {int(overlap_n*100)}%, H: {round(y_max_sw_n,2)}',s=40)
    ax1.plot([x_max_sw_n,x_max_sw_n],[0,y_max_sw_n],c='g')
    
    ax1.legend()
    ax1.set_title(f'{def_para}')
    plt.tight_layout()

#     fig_STORM_cross_section_topfit.savefig(results_folder+f'fig_STORM_cross_section_topfit_{def_para}.png',dpi=600)

    return height_polyfit,height_bin,sliding_window_height,STORM_Data_upper_half

def filter_effect(fm, Nf, LLRf, epsf, msf,locS_c,locS):
    fig, ax =plt.subplots(2,2,figsize=(8,8))
    locS_c = locS_c[locS_c['N']>Nf]  
    ax[0][0].scatter(locS['X'],locS['Y'], s=0.5, alpha=0.5,c='grey')
    ax[0][0].scatter(locS_c['X'],locS_c['Y'], s=1, alpha=0.8,c='C0')


    locS_c = locS_c[locS_c['LLR']<LLRf] 
    ax[0][1].scatter(locS['X'],locS['Y'], s=0.5, alpha=0.5,c='grey')
    ax[0][1].scatter(locS_c['X'],locS_c['Y'], s=1, alpha=0.8,c='C0')


    locS_c = locS_c.sort_values('X')
    x_locS = locS_c.loc[:, ['X','Y']].values
    dbscanS = DBSCAN(eps = epsf,
                     min_samples = msf).fit(x_locS) # fitting the model
    labelsS = dbscanS.labels_# getting the labels
    locS_c['cluster2']=labelsS
    locS_c = locS_c[locS_c['cluster2']>=0]

    # filter for clustering
    ax[1][0].scatter(locS['X'],locS['Y'], s=0.5, alpha=0.5,c='grey')
    ax[1][0].scatter(x_locS[:, 0], x_locS[:,1], c = labelsS, s=1) # plotting the clusters



    # Calculate the heights with fitting polynome:
#     upper_cutoff_poly = locS_c.copy()
#     upper_cutoff_poly = filt(upper_cutoff_poly,'X',-10,10)
#     upper_cutoff_poly = min(upper_cutoff_poly['Y'])
    upper_cutoff_poly = 100
    x_fit,h_poly,x_u_fit,y_u_fit,S_D_u_half = calc_height_polynome(locS_c,upper_cutoff_poly)

    # fit the polynome
    ax[1][1].scatter(S_D_u_half['X'],S_D_u_half['Y'],s=0.8,alpha=0.9,c='C0') 
    ax[1][1].scatter(locS_c['X'],locS_c['Y'], s=0.5, alpha=0.5,c='grey')
    ax[1][1].plot(x_u_fit,y_u_fit,color='red')
    ax[1][1].plot([x_fit,x_fit],[0,h_poly],color='black',
                label = f'height_polyfit: {round(h_poly,2)}')

    ax[0][0].set_title(f'N: {Nf}')
    ax[0][1].set_title(f'LLR: {LLRf}')
    ax[1][0].set_title(f'eps:{epsf}, cluster min: {msf}')
    ax[1][1].set_title(f'Height: {np.round(h_poly,2)} nm')

    ax[0][0].set_xlabel("X / nm"),ax[0][0].set_ylabel("Z / nm")
    ax[0][1].set_xlabel("X / nm"),ax[0][1].set_ylabel("Z / nm")
    ax[1][0].set_xlabel("X / nm"),ax[1][0].set_ylabel("Z / nm")
    ax[1][1].set_xlabel("X / nm"),ax[1][1].set_ylabel("Z / nm")

    ax[0][0].axis('equal'), ax[0][1].axis('equal')
    ax[1][0].axis('equal'), ax[1][1].axis('equal')

    ax[1][1].axhline(color='k',alpha=0.2)
    ax[1][1].axvline(color='k',alpha=0.2)
    plt.tight_layout()
    fig.savefig(filter_folder+f'/filter_N{Nf}_LLR{LLRf}_eps{epsf}_msf{msf}_H{h_poly}.png',dpi=600)
    new_row = {'fm':fm,'N':Nf,'LLR':LLRf,'eps':epsf,'min':msf,'H':h_poly}
    return new_row

def calc_height_polynome(locS_c,upper_cutoff=None):
    # Set default value inside the function
    if upper_cutoff is None:
        upper_cutoff = max(locS_c['Y']) / 2
    STORM_Data_upper_half = locS_c[locS_c['Y']>upper_cutoff]

    # make the fit for the upper data points
    upper_fit = np.polyfit(STORM_Data_upper_half['X'],STORM_Data_upper_half['Y'],2)
    x_upper_fit = np.linspace(-100,100,1000)
    y_upper_fit = np.polyval(upper_fit,x_upper_fit)

    # save the upper point:
    height_polyfit = max(y_upper_fit)
    x_STORM_fit_max = x_upper_fit[list(y_upper_fit).index(max(y_upper_fit))]
    return x_STORM_fit_max,height_polyfit,x_upper_fit,y_upper_fit,STORM_Data_upper_half



In [3]:
# Used functions for binning:

## Use a sliding windw to make the binning:
   
# Define a sliding window with the beginning a, the end b and the windw size ws
# Then define how much the first window overlaps with the next defined as overlap
# Filter out the data in that window and calculate whatever you are interested in

# overlap = 0.5 by default
# 0.2 would be 20 % overlap of two windows
# 1 would be no overlap and 0 is not a valid input

def sliding_win(df, ws, overlap):
    start, end = min(df['X']), max(df['X'])
    res = []
    a = start
    step = ws * (1 - overlap)  # Step size considering overlap
    
    print(a,end)
    while a < end:
        b = a + ws
        dfs = filt(df, 'X', a, b)
        if len(dfs) > 0:  # Ensure there is data in the window
            H = np.median(dfs['Y'])  # Assuming 'Y' is the column of interest
            res.append([np.mean([a, b]), H])
        a += step  # Move the window
    y_values = [r[1] for r in res]
    max_index = y_values.index(max(y_values))
    res_max = res[max_index]
    
    x_values = [k[0] for k in res]
    y_values = [k[1] for k in res]
    res = [x_values,y_values]
    return res,res_max[0],res_max[1]
    # return all binned datapoints and also where the max is
    
def sliding_win_n(df, n, overlap):
    
#     Applies a sliding window over the dataframe with a fixed number of data points per window.

#     Parameters:
#     df (pd.DataFrame): DataFrame containing 'X' and 'Y' columns.
#     n (int): Number of data points in each window.
#     overlap (float): Overlap fraction (0 to 1), where 0 means no overlap, 1 means full overlap.

#     Returns:
#     list: List of [mean_x, median_y] for each window.
#     float: x-value where Y is maximum.
#     float: maximum Y value.
    res = []
    step = int(n * (1 - overlap))  # Calculate step size in terms of data points

    if step < 1:  # Avoid infinite loops
        step = 1

    for i in range(0, len(df) - n + 1, step):
        window = df.iloc[i:i+n]  # Select n data points
        mean_x = np.mean(window['X'])  # Compute mean X value for the window
        median_y = np.median(window['Y'])  # Compute median Y value for the window
        res.append([mean_x, median_y])

    # Find the max Y value and its corresponding X
    y_values = [r[1] for r in res]
    max_index = y_values.index(max(y_values))
    res_max = res[max_index]
    
    x_values = [k[0] for k in res]
    y_values = [k[1] for k in res]
    res = [x_values,y_values]
    return res, res_max[0], res_max[1]  # Return all bins and max location

# Perform "normal" binning without any overlap:
def bin_data_by_x(df, bin_width):
#     Bins the data along the X-axis into equal-width bins, calculates the median for each bin, 
#     and returns the binned data along with the corresponding Y values.

#     Parameters:
#     df (pd.DataFrame): DataFrame with 'X' and 'Y' columns.
#     bin_width (float): Width of each bin.

#     Returns:
#     DataFrame: DataFrame with the median values for each bin.
#     list: List of Y values (median of each bin).
    
    df = df.sort_values('X')
    
    # Calculate the range of X values
    X_range = max(df['X']) - min(df['X'])
    
    # Determine the number of bins based on X_range and bin_width
    num_of_bins = int(X_range / bin_width)
    
    # Create the bins along the X-axis
    c_data = pd.cut(df['X'], bins=num_of_bins)
    
    # Add the 'bin' column to the DataFrame
    df['bin'] = c_data
    
    # Group by the 'bin' column and calculate the median of Y for each bin
    df_mean = df.groupby(['bin']).median()
    
    # Extract the median Y values
    max_y_index = df_mean['Y'].idxmax() 
    mean_height_x = df_mean.loc[max_y_index, 'X'] 
    mean_height = df_mean.loc[max_y_index, 'Y']
    
    return df_mean, mean_height_x, mean_height




In [6]:
# For calculating the error:
def sliding_win_height(df, n, overlap):
    
#     Applies a sliding window over the dataframe with a fixed number of data points per window.

#     Parameters:
#     df (pd.DataFrame): DataFrame containing 'X' and 'Y' columns.
#     n (int): Number of data points in each window.
#     overlap (float): Overlap fraction (0 to 1), where 0 means no overlap, 1 means full overlap.

#     Returns:
#     list: List of [mean_x, median_y] for each window.
#     float: x-value where Y is maximum.
#     float: maximum Y value.
    res = []
    step = int(n * (1 - overlap))  # Calculate step size in terms of data points

    if step < 1:  # Avoid infinite loops
        step = 1

    for i in range(0, len(df) - n + 1, step):
        window = df.iloc[i:i+n]  # Select n data points
        mean_x = np.mean(window['X'])  # Compute mean X value for the window
        median_y = np.median(window['Y'])  # Compute median Y value for the window
        res.append([mean_x, median_y])

    # Find the max Y value and its corresponding X
    y_values = [r[1] for r in res]
    max_index = y_values.index(max(y_values))
    res_max = res[max_index]
    
    x_values = [k[0] for k in res]
    y_values = [k[1] for k in res]
    res = [x_values,y_values]
    
    STORM_sw_H_n,x_max_sw_n,y_max_sw_n = res,res_max[0], res_max[1]
    return STORM_sw_H_n, x_max_sw_n, y_max_sw_n  # Return all bins and max location


def calc_error(T_test,n_loc,overlap):
#     T_test...profile with X and Y values
#     n_loc...number of locs in sliding window
#     overlap...number of overlapping locs in sliding window
    
    error_height_boot = [] # bootstrapping
    error_height_resa = [] # subsampling

    num_of_datapoints = len(T_test)
    low, high = 0, num_of_datapoints  # Change these to your desired range

    
    # Bootstrapping
    for k in range(10000):

        # Generate random numbers in the range [low, high)
        r_num_boot = np.random.choice(range(low, high), 
                                      size = num_of_datapoints, replace=True)
   
        T_test_resample_boot = T_test.iloc[r_num_boot]
        T_test_resample_boot = T_test_resample_boot.sort_values('X')
        H_n_boot, x_n_boot, y_max_boot = sliding_win_height(T_test_resample_boot,n_loc,overlap)
        error_height_boot += [y_max_boot]
    
    # Subsampling
    for j in range(1000):
        r_num_resa = np.random.choice(range(low, high), 
                                      size = int(num_of_datapoints/2), 
                                      replace=False)
        
        T_test_resample_resa = T_test.iloc[r_num_resa]
        T_test_resample_resa = T_test_resample_resa.sort_values('X')
        H_n_resa, x_n_resa, y_max_resa = sliding_win_height(T_test_resample_resa,n_loc,overlap)
        error_height_resa += [y_max_resa]

    med_boot = np.mean(error_height_boot)
    sdt_boot = np.std(error_height_boot)

    med_resa = np.mean(error_height_resa)
    sdt_resa = np.std(error_height_resa)

    return error_height_boot,med_boot,sdt_boot,error_height_resa,med_resa,sdt_resa



def extract_n(df_plot,H_AFM_compare):
    # make a linear fit through the datapoints:
    x_fit = df_plot['refractive_index']
    y_fit = df_plot['sliding_window_height']
    fit_p = np.polyfit(x_fit,y_fit,1)
    x_n = np.linspace(min(x_fit),max(x_fit),100)
    y_n = np.polyval(fit_p,x_n)

    # Calculate the intersection:
    n = (H_AFM_compare-fit_p[1])/fit_p[0]
    return n